# Simple result overview on TC displacement risk

This notebook explores a few crude result outputs from the TC displacement risk computations

In [2]:
import pandas as pd
import glob
import os

path_results = '/Users/evelynm/Documents/UNU_IDMC/data/risk/TC'# '/cluster/work/climate/evelynm/IDMC_UNU/results/risk_tc/'
path_bem = '/cluster/work/climate/evelynm/IDMC_UNU/exposure/bem_cntry_files/'

In [2]:
all_iso3 = [cntry[0:3].upper() for cntry in os.listdir(path_bem)]

In [3]:
iso='TGO'

In [14]:
df_list=[]
for ssp in [245, 370, 585]:
    try:
        df_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_0.55_TC.csv', nrows=3))
    except:
        continue

In [22]:
cntry_df = pd.concat(df_list, axis=0).reset_index(drop=True)
cntry_df['iso3'] = iso
cntry_df.pop('Unnamed: 0')
cntry_df.pop('Exposure')
cntry_df

,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,iso3
0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
1,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,TGO
2,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,TGO
3,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
4,2050,ssp370,0.0,0.0,0.0,0.0,0.0,0.0,TGO
5,2100,ssp370,0.0,0.0,0.0,0.0,0.0,0.0,TGO
6,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
7,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,TGO
8,2100,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,TGO


In [31]:
global_list = []
for iso in all_iso3:
    cntry_list = []
    if os.path.exists(path_results+iso):
        for ssp in [245, 370, 585]:
            try:
                cntry_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_0.55_TC.csv', nrows=3))
                cntry_df = pd.concat(cntry_list, axis=0).reset_index(drop=True)
                cntry_df['iso3'] = iso
                cntry_df.pop('Unnamed: 0')
                cntry_df.pop('Exposure')
            except:
                continue
        global_list.append(cntry_df)

In [32]:
global_df = pd.concat(global_list, axis=0).reset_index(drop=True)
global_df

,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,iso3
0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
2,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
3,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
4,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,RUS
...,...,...,...,...,...,...,...,...,...
1366,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1367,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1368,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1369,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,RUS


In [33]:
global_df.to_csv(path_results+'TC_displ_admin0_med.csv')

### Rename admin1 to GADM classifications for better map plotting

In [ ]:
import os
import pandas as pd

gadm_attrs = pd.read_csv('/Users/evelynm/Documents/UNU_IDMC/data/exposure/attribute_table_gadm41.csv')
gadm_attrs_map = dict(zip(gadm_attrs.fid, gadm_attrs.GID_1))
gadm_attrs_map[-999] = 'admin0'
    
path_results = '/Users/evelynm/Documents/UNU_IDMC/data/risk/TC'
isos3 = os.listdir(path_results)

for iso3 in isos3:
    result_paths = os.listdir(path_results+'/'+iso3)
    for result_path in result_paths:
        try:
            df_results = pd.read_csv(path_results+'/'+iso3+'/'+result_path)
            df_results.replace({'admin0':'admin_-999'}, inplace=True)
            df_results['admin1'] = df_results.Exposure.apply(lambda x: x.split('_')[-1]).astype(int)
            df_results['GID_1'] = df_results.admin1.map(gadm_attrs_map)
            df_results.to_csv(path_results+'/'+iso3+'/'+result_path)
        except:
            print(iso3)

## Generate one single csv-File for admin-1

In [4]:
isos3 = os.listdir(path_results)

In [30]:
ssps = ['ssp245', 'ssp370','ssp585']

dict_scen = {'ssp245':'optimistic',
             'ssp370' : 'medium',
             'ssp585' : 'pessimistic',
             'ERA-5' : 'current'
            }

threshs = [0.3, 0.55, 0.7]
dict_thresh = {0.3 : 'high',
               0.55 : 'medium',
               0.7 : 'low'}


In [60]:
df_list = []
for iso3 in isos3:
        for ssp in ssps:
                try:
                    df03 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.3_TC.csv')
                    df03 = df03.iloc[3:,:]
                    for col in df03.columns:
                        df03.rename({col:f'{col}_{dict_thresh[0.3]}'}, axis=1, inplace=True)
                    
                    df05 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.55_TC.csv')
                    df05 = df05.iloc[3:,:]
                    for col in df05.columns:
                        df05.rename({col:f'{col}_{dict_thresh[0.55]}'}, axis=1, inplace=True)
                        
                    df07 = pd.read_csv(f'{path_results}/{iso3}/{iso3}_{ssp}_0.7_TC.csv')
                    df07 = df07.iloc[3:,:]
                    for col in df07.columns:
                        df07.rename({col:f'{col}_{dict_thresh[0.7]}'}, axis=1, inplace=True)
                except:
                    continue
                df_iso = pd.concat([df03[['Exposure_high','Period_high','Scenario_high','AAD_high','RP_10_high','RP_25_high','RP_50_high','RP_100_high','RP_250_high','admin1_high','GID_1_high']],
                                       df05[['Exposure_medium','Period_medium','Scenario_medium','AAD_medium','RP_10_medium','RP_25_medium','RP_50_medium','RP_100_medium','RP_250_medium']],
                                        df07[['Exposure_low','Period_low','Scenario_low','AAD_low','RP_10_low','RP_25_low','RP_50_low','RP_100_low','RP_250_low']]], 
                                                 axis=1)
                if 'GID_1' in df_iso.columns:
                    df_iso['GID_1'] = df_iso.GID_1.fillna(iso3)
                df_list.append(df_iso)


In [61]:
df_concat = pd.concat(df_list)

In [62]:
# delete unncecessary columns:
for col in df_concat:
    if 'Unnamed' in col:
        df_concat.pop(col)
for col in ['Exposure_high', 'Exposure_low','Exposure_medium', 'GID_1_medium', 'Period_high', 'Period_medium', 'Scenario_high', 'Scenario_medium']:
    if col in df_concat.columns:
        df_concat.pop(col)

In [63]:
df_concat

,AAD_high,RP_10_high,RP_25_high,RP_50_high,RP_100_high,RP_250_high,admin1_high,GID_1_high,AAD_medium,RP_10_medium,...,RP_100_medium,RP_250_medium,Period_low,Scenario_low,AAD_low,RP_10_low,RP_25_low,RP_50_low,RP_100_low,RP_250_low
3,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,435,CAN.2_1,0.0,0.0,...,0.0,0.0,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,437,CAN.4_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,437,CAN.4_1,0.0,0.0,...,0.0,0.0,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.0,0.0,0.0,0.0,3503,VEN.21_1,0.0,0.0,...,0.0,0.0,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,3503,VEN.21_1,0.0,0.0,...,0.0,0.0,2100,ssp585,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,0.0,0.0,0.0,0.0,3507,VEN.25_1,0.0,0.0,...,0.0,0.0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,3507,VEN.25_1,0.0,0.0,...,0.0,0.0,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df_concat.rename({'admin1_high':'fid'}, axis=1, inplace=True)
df_concat.rename({'GID_1_high':'GID_1'}, axis=1, inplace=True)
df_concat.rename({'Scenario_low':'scenario'}, axis=1, inplace=True)
df_concat.rename({'Period_low':'time_horzion'}, axis=1, inplace=True)

In [65]:
df_concat.replace(dict_scen, inplace=True)

In [69]:
for col in df_concat.columns:
    if 'RP' in col:
        rp, vuln = col.split('_')[-2:]
        df_concat.rename({col:f'PMD_{rp}_{vuln}'}, axis=1, inplace=True)
df_concat.columns

Index(['AAD_high', 'PMD_10_high', 'PMD_25_high', 'PMD_50_high', 'PMD_100_high',
       'PMD_250_high', 'fid', 'GID_1', 'AAD_medium', 'PMD_10_medium',
       'PMD_25_medium', 'PMD_50_medium', 'PMD_100_medium', 'PMD_250_medium',
       'time_horzion', 'scenario', 'AAD_low', 'PMD_10_low', 'PMD_25_low',
       'PMD_50_low', 'PMD_100_low', 'PMD_250_low'],
      dtype='object')

In [70]:
df_concat = df_concat[['fid', 'GID_1','time_horzion', 'scenario','AAD_high', 'PMD_10_high', 'PMD_25_high', 'PMD_50_high', 'PMD_100_high',
       'PMD_250_high', 'AAD_medium', 'PMD_10_medium',
       'PMD_25_medium', 'PMD_50_medium', 'PMD_100_medium', 'PMD_250_medium',
        'AAD_low', 'PMD_10_low', 'PMD_25_low',
       'PMD_50_low', 'PMD_100_low', 'PMD_250_low']]

In [71]:
df_concat

,fid,GID_1,time_horzion,scenario,AAD_high,PMD_10_high,PMD_25_high,PMD_50_high,PMD_100_high,PMD_250_high,...,PMD_25_medium,PMD_50_medium,PMD_100_medium,PMD_250_medium,AAD_low,PMD_10_low,PMD_25_low,PMD_50_low,PMD_100_low,PMD_250_low
3,435,CAN.2_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,435,CAN.2_1,2050,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,435,CAN.2_1,2100,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,437,CAN.4_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,437,CAN.4_1,2050,optimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,3503,VEN.21_1,2050,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,3503,VEN.21_1,2100,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,3507,VEN.25_1,hist,current,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,3507,VEN.25_1,2050,pessimistic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
df_concat.to_csv(path_results+'/displacement_risk_tc_global_admin1.csv')